In [60]:
import torch
import numpy as np
from torch import FloatTensor, LongTensor
from typing import Tuple


In [63]:
def loc_ref_frame(pos: FloatTensor, center: FloatTensor, radius: float) -> Tuple[FloatTensor, FloatTensor, FloatTensor]:
    """"""
    rel_pos = pos - center # relative position
    rel_pos_norm = torch.norm(rel_pos, dim = 1) # tensor of position vector norm
    
    # Modified covariance matrix estimator
    weight = (radius - rel_pos_norm)
    weight_sum = sum(weight)
    M = torch.mm(weight * torch.t(rel_pos), rel_pos) / weight_sum
    print(M)
    
    # Eigenvalues and eigenvectors calculation
    # Vectors as columns, ordered by ascending e_val's
    e_val, e_vec = torch.symeig(M, eigenvectors = True)
    print(e_val)
    
    # Egien vectors sense calculation
    # x and z denote respectively e_vec of highest and e_vec of lowest e_val.
    S_T = torch.mm(pos, e_vec) 
    S_x, S_z = S_T[:, 2], S_T[:, 0]
    
    # Sign of x 
    if (S_x >= 0).nonzero().size()[0] >= num_points / 2: 
        sign_x = 1
    else: sign_x = -1
    
    # Sign of z
    if (S_z >= 0).nonzero().size()[0] >= num_points / 2:
        sign_z = 1
    else: sign_z = -1
        
    # LRF vectors
    x = sign_x * e_vec[:, 2]  
    z = sign_z * e_vec[:, 0]
    y = torch.cross(x, z)
    
    return x, y, z

In [64]:
num_points = 10000
radius = 3.
center = np.array([1., 2., 2.])


# Sample points (num_points) according to uniform distribution
# on the ball of the center and the radius given above

pos = np.random.randn(num_points, 3)
rad = np.random.uniform(size = (num_points, 1))
pos_norm = np.array(np.linalg.norm(pos, axis = 1)).reshape(num_points, 1)
pos = pos / pos_norm * np.sqrt(rad) * radius  # Uniform distribution on ball
#pos *= np.array([1., 0.5, 0.1]) # Uniform distribution on elipsiod
pos += center


# Converting into Tensors
pos_T = torch.from_numpy(pos)
center_T = torch.from_numpy(center)

# Test
result = loc_ref_frame(pos_T, center_T, radius)

tensor([[ 0.8900,  0.0021, -0.0100],
        [ 0.0021,  0.8766, -0.0031],
        [-0.0100, -0.0031,  0.8996]], dtype=torch.float64)
tensor([0.8761, 0.8838, 0.9064], dtype=torch.float64)


In [2]:
import pcl
p = pcl.load("C/table_scene_lms400.pcd")

ImportError: libpcl_keypoints.so.1.7: cannot open shared object file: No such file or directory